# Ollama + OpenAI + Python

## 1. मॉडल का नाम निर्दिष्ट करें

यदि आपने "phi3:mini" के अलावा कोई अन्य मॉडल उपयोग किया है, तो नीचे दिए गए सेल में मान बदलें। यह वेरिएबल पूरे नोटबुक में कोड में उपयोग किया जाएगा।


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AI क्लाइंट सेटअप करें

आमतौर पर OpenAI क्लाइंट का उपयोग OpenAI.com या Azure OpenAI के साथ बड़े भाषा मॉडल्स के साथ बातचीत करने के लिए किया जाता है।  
हालांकि, इसे Ollama के साथ भी उपयोग किया जा सकता है, क्योंकि Ollama "http://localhost:11434/v1" पर OpenAI-संगत एंडपॉइंट प्रदान करता है।


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. चैट पूरा करें

अब हम OpenAI SDK का उपयोग करके बातचीत के लिए एक प्रतिक्रिया उत्पन्न कर सकते हैं। यह अनुरोध बिल्लियों के बारे में एक हाइकू उत्पन्न करना चाहिए:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. प्रॉम्प्ट इंजीनियरिंग

भाषा मॉडल को भेजा गया पहला संदेश "सिस्टम संदेश" या "सिस्टम प्रॉम्प्ट" कहलाता है, और यह मॉडल के लिए समग्र निर्देश सेट करता है।  
आप अपना कस्टम सिस्टम प्रॉम्प्ट प्रदान कर सकते हैं ताकि भाषा मॉडल को अलग तरीके से आउटपुट उत्पन्न करने के लिए निर्देशित किया जा सके।  
नीचे दिए गए `SYSTEM_MESSAGE` को संशोधित करें ताकि मॉडल आपके पसंदीदा प्रसिद्ध फिल्म/टीवी किरदार की तरह जवाब दे, या अन्य सिस्टम प्रॉम्प्ट के लिए [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) से प्रेरणा लें।  

एक बार जब आपने सिस्टम संदेश को कस्टमाइज़ कर लिया है, तो `USER_MESSAGE` में पहला उपयोगकर्ता प्रश्न प्रदान करें।


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. कुछ उदाहरण

भाषा मॉडल को निर्देशित करने का एक और तरीका है "कुछ उदाहरण" प्रदान करना, यानी प्रश्न/उत्तर की एक श्रृंखला जो यह दिखाती है कि इसे कैसे प्रतिक्रिया देनी चाहिए।

नीचे दिए गए उदाहरण में, भाषा मॉडल को एक शिक्षण सहायक (टीचिंग असिस्टेंट) की तरह कार्य करने के लिए प्रेरित किया गया है। इसमें कुछ प्रश्न और उत्तर दिए गए हैं जो एक टीए (TA) दे सकता है, और फिर मॉडल को एक ऐसा प्रश्न दिया गया है जो एक छात्र पूछ सकता है।

पहले इसे आज़माएं, और फिर `SYSTEM_MESSAGE`, `EXAMPLES`, और `USER_MESSAGE` को एक नए परिदृश्य के लिए संशोधित करें।


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. पुनर्प्राप्ति संवर्धित जनरेशन

RAG (पुनर्प्राप्ति संवर्धित जनरेशन) एक तकनीक है जो किसी विशेष क्षेत्र के लिए भाषा मॉडल को सटीक उत्तर देने में सक्षम बनाती है। इसमें पहले ज्ञान स्रोत से प्रासंगिक जानकारी प्राप्त की जाती है और फिर उस जानकारी के आधार पर उत्तर तैयार किया जाता है।

हमने हाइब्रिड कारों के बारे में डेटा के साथ एक स्थानीय CSV फ़ाइल प्रदान की है। नीचे दिया गया कोड CSV फ़ाइल को पढ़ता है, उपयोगकर्ता के प्रश्न से मेल खाने वाली जानकारी खोजता है, और फिर प्राप्त जानकारी के आधार पर उत्तर तैयार करता है। ध्यान दें कि यह पिछले उदाहरणों की तुलना में अधिक समय लेगा, क्योंकि यह मॉडल को अधिक डेटा भेजता है। यदि आप देख रहे हैं कि उत्तर अभी भी डेटा में आधारित नहीं है, तो आप सिस्टम इंजीनियरिंग का प्रयास कर सकते हैं या अन्य मॉडलों का उपयोग कर सकते हैं। सामान्यतः, RAG बड़े मॉडलों या SLMs के फाइन-ट्यून किए गए संस्करणों के साथ अधिक प्रभावी होता है।


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
